In [122]:
import copy
import numpy as np
import pandas as pd

In [123]:
debug = False
max_iter = 10_000
results = []
n_sizes = [3, 5, 7, 10, 15, 20]

In [124]:
def gen_hilbert_matrix(n):
    H = [[1 / (i + j + 1) for j in range(n)] for i in range(n)]
    b = [sum(row) for row in H]
    return H, b


def gen_matrix(n):
    J = [[0 for _ in range(n)] for _ in range(n)]
    b = [1] * n
    for i in range(n):
        for j in range(n):
            if i == j:
                J[i][j] = 2
            elif abs(i - j) == 1:
                J[i][j] = -1
            else:
                J[i][j] = 0
    return J, b


def euclidean_dist(x1, x2):
    return np.linalg.norm(np.array(x1) - np.array(x2))


def inf_norm(x1, x2):
    return np.linalg.norm(np.array(x1) - np.array(x2), ord=np.inf)


def l1_norm(x1, x2):
    return np.linalg.norm(np.array(x1) - np.array(x2), ord=1)


def residual_norm(A, x, b):
    A_np = np.array(A, dtype=float)
    x_np = np.array(x, dtype=float)
    b_np = np.array(b, dtype=float)
    return np.linalg.norm(A_np @ x_np - b_np)

In [125]:
def gauss_elimination(A, b, debug=False):
    if debug:
        print("==============================")
        print("Gaussian Elimination Method")
        print(f"\nMatrix A: {A}")
        print(f"\nVector b: {b}")

    n = len(A)

    # Forward pass of elimination
    for i in range(n-1):
        for j in range(i+1, n):
            k_temp = - A[j][i] / A[i][i]
            if debug:
                print(f"Step {i+1} {j+1}: k = {k_temp:.2f}")

            temp_row = [a * k_temp for a in A[i]]
            if debug:
                print(f"Temp row: {temp_row}")

            for k in range(n):
                A[j][k] += temp_row[k]

            b[j] += k_temp * b[i]

            if debug:
                print(f"Updated A = {A}")
                print(f"Updated b = {b}")

    x_sol = [None for _ in range(n)]
    for i in range(n):
        row_idx = n - i - 1
        res = b[row_idx]
        for j in range(n - i, n):
            res -= A[row_idx][j] * x_sol[j]
        x_sol[row_idx] = res / A[row_idx][row_idx]
        if debug:
            print(f"x_{n-i-1} = {x_sol[n-i-1]:.2f}")

    if debug:
        print_x = [f"{xi:.2f}" for xi in x_sol]
        print(f"\nSolution: {print_x}")
        print("==============================\n")

    return x_sol


def jacobi_iter(A, b, x0=None, eps=1e-10, max_iter=100, debug=False):
    if debug:
        print("==============================")
        print("Jacobi Iteration Method")
        print(f"Matrix A: {A}")
        print(f"Vector b: {b}\n")

    len_A = len(A)
    if x0 is None:
        x0 = [0.0] * len_A
    x = x0.copy()

    # x1(k+1) = (b1 - SUM(a12 * x2(k) + ... )) / a11
    # x2(k+1) = (b2 - SUM(a21 * x1(k) + ... )) / a22
    # ...
    for k in range(max_iter):
        x_new = [0.0] * len_A

        for i in range(len_A):
            sum_ax = 0.0
            for j in range(len_A):
                if j != i:
                    sum_ax += A[i][j] * x[j]
            x_new[i] = (b[i] - sum_ax) / A[i][i]

        vec_dist = euclidean_dist(x_new, x)

        if debug:
            print_x = [f'{xi:.6f}' for xi in x_new]
            print(f"Iteration {k+1}: x = {print_x}")

        if not np.isfinite(vec_dist):
            return x_new, k+1

        if vec_dist < eps:
            print("Converged!")
            return x_new, k+1

        x = x_new.copy()

    if debug:
        print_x = [f"{xi:.2f}" for xi in x]
        print(f"\nSolution: {print_x}")
        print("==============================\n")

    return x, max_iter


def seidel_iter(A, b, x0=None, eps=1e-10, max_iter=100, debug=False):
    if debug:
        print("==============================")
        print("Gauss-Seidel Iteration Method")
        print(f"Matrix A: {A}")
        print(f"Vector b: {b}\n")

    len_A = len(A)
    if x0 is None:
        x0 = [0.0] * len_A
    x = x0.copy()

    # x1(k+1) = (b1 - a12 * x2(k)) / a11
    # x2(k+1) = (b2 - a21 * x1(k+1)) / a22
    # ...
    for k in range(max_iter):
        x_new = [0.0] * len_A

        for i in range(len_A):
            sum_ax = 0.0
            for j in range(len_A):
                if j != i:
                    if j < i:
                        sum_ax += A[i][j] * x_new[j]
                    else:
                        sum_ax += A[i][j] * x[j]
            x_new[i] = (b[i] - sum_ax) / A[i][i]

        vec_dist = euclidean_dist(x_new, x)

        if debug:
            print_x = [f'{xi:.6f}' for xi in x_new]
            print(f"Iteration {k+1}: x = {print_x}")

        if vec_dist < eps:
            print("Converged!")
            return x_new, k+1

        x = x_new.copy()

    if debug:
        print_x = [f"{xi:.2f}" for xi in x]
        print(f"\nSolution: {print_x}")
        print("==============================\n")

    return x, max_iter

In [126]:
norm_fn = inf_norm

In [127]:
print("HILBERT MATRIX")

for n in n_sizes:
    H, b = gen_hilbert_matrix(n)
    x_exact = [1.0] * n

    # Gaussian Elimination
    A_copy, b_copy = copy.deepcopy(H), copy.deepcopy(b)
    x_gauss = gauss_elimination(A_copy, b_copy, debug=debug)
    err_gauss = norm_fn(x_gauss, x_exact)
    res_gauss = residual_norm(H, x_gauss, b)
    results.append({
        'Matrix': 'Hilbert',
        'Dimension': n,
        'Method': 'Gaussian Elimination',
        'Abs.Error': err_gauss,
        'Residual': res_gauss,
        'Iterations': None
    })

    # Jacobi Iteration
    A_copy, b_copy = copy.deepcopy(H), copy.deepcopy(b)
    x_jacobi, iters_jacobi = jacobi_iter(A_copy, b_copy, max_iter=max_iter, debug=debug)
    err_jacobi = norm_fn(x_jacobi, x_exact)
    res_jacobi = residual_norm(H, x_jacobi, b)
    results.append({
        'Matrix': 'Hilbert',
        'Dimension': n,
        'Method': 'Jacobi Iteration',
        'Abs.Error': err_jacobi,
        'Residual': res_jacobi,
        'Iterations': f'{iters_jacobi:.0f}'
    })

    # Gauss-Seidel Iteration
    A_copy, b_copy = copy.deepcopy(H), copy.deepcopy(b)
    x_seidel, iters_seidel = seidel_iter(A_copy, b_copy, max_iter=max_iter, debug=debug)
    err_seidel = norm_fn(x_seidel, x_exact)
    res_seidel = residual_norm(H, x_seidel, b)
    results.append({
        'Matrix': 'Hilbert',
        'Dimension': n,
        'Method': 'Gauss-Seidel Iteration',
        'Abs.Error': err_seidel,
        'Residual': res_seidel,
        'Iterations': f'{iters_seidel:.0f}'
    })

HILBERT MATRIX
Converged!


In [128]:
print("TRIDIAGONAL MATRIX")

for n in n_sizes:
    J, b = gen_matrix(n)
    x_exact = np.linalg.solve(
        np.array(J, dtype=float),
        np.array(b, dtype=float)
    ).tolist()

    # Gaussian Elimination
    A_copy, b_copy = copy.deepcopy(J), copy.deepcopy(b)
    x_gauss = gauss_elimination(A_copy, b_copy, debug=debug)
    err_gauss = norm_fn(x_gauss, x_exact)
    res_gauss = residual_norm(J, x_gauss, b)
    results.append({
        'Matrix': 'Tridiagonal',
        'Dimension': n,
        'Method': 'Gaussian Elimination',
        'Abs.Error': err_gauss,
        'Residual': res_gauss,
        'Iterations': None
    })

    # Jacobi Iteration
    A_copy, b_copy = copy.deepcopy(J), copy.deepcopy(b)
    x_jacobi, iters_jacobi = jacobi_iter(A_copy, b_copy, max_iter=max_iter, debug=debug)
    err_jacobi = norm_fn(x_jacobi, x_exact)
    res_jacobi = residual_norm(J, x_jacobi, b)
    results.append({
        'Matrix': 'Tridiagonal',
        'Dimension': n,
        'Method': 'Jacobi Iteration',
        'Abs.Error': err_jacobi,
        'Residual': res_jacobi,
        'Iterations': f'{iters_jacobi:.0f}'
    })

    # Gauss-Seidel Iteration
    A_copy, b_copy = copy.deepcopy(J), copy.deepcopy(b)
    x_seidel, iters_seidel = seidel_iter(A_copy, b_copy, max_iter=max_iter, debug=debug)
    err_seidel = norm_fn(x_seidel, x_exact)
    res_seidel = residual_norm(J, x_seidel, b)
    results.append({
        'Matrix': 'Tridiagonal',
        'Dimension': n,
        'Method': 'Gauss-Seidel Iteration',
        'Abs.Error': err_seidel,
        'Residual': res_seidel,
        'Iterations': f'{iters_seidel:.0f}'
    })

TRIDIAGONAL MATRIX
Converged!
Converged!
Converged!
Converged!
Converged!
Converged!
Converged!
Converged!
Converged!
Converged!
Converged!
Converged!


In [129]:
df = pd.DataFrame(results)
pd.set_option('display.float_format', '{:.2e}'.format)
df

,Matrix,Dimension,Method,Abs.Error,Residual,Iterations
0,Hilbert,3,Gaussian Elimination,7.88e-15,2.22e-16,NaN
1,Hilbert,3,Jacobi Iteration,9.72e+153,inf,651
2,Hilbert,3,Gauss-Seidel Iteration,3.70e-09,2.17e-11,957
3,Hilbert,5,Gaussian Elimination,2.59e-12,0.00e+00,NaN
4,Hilbert,5,Jacobi Iteration,6.43e+153,1.20e+154,286
5,Hilbert,5,Gauss-Seidel Iteration,2.16e-02,1.14e-07,10000
6,Hilbert,7,Gaussian Elimination,3.28e-09,5.55e-16,NaN
7,Hilbert,7,Jacobi Iteration,2.59e+154,inf,216
8,Hilbert,7,Gauss-Seidel Iteration,1.24e-02,2.21e-07,10000
9,Hilbert,10,Gaussian Elimination,5.80e-04,2.94e-16,NaN
